__OPerator Overloading:__ Intecept built-in operations in class methods 

In [ ]:
'''When used these methods allow classes to emulate the interfaces
of built-in objects, and so appear more consistent.'''
%-------------------------------------------------------------------------
# __init__ (constructor) and __sub__ (allows substraction of objects):

class Number:
    def __init__(self, start): # On Number(start)
        self.data = start
    def __sub__(self, other): # On instance - other--> its contrary is __add__
        return Number(self.data - other) # Result is a new instance

>>> from number import Number # Fetch class from module
>>> X = Number(5) # Number.__init__(X, 5)
>>> Y = X - 2 # Number.__sub__(X, 2)
>>> Y.data # Y is new Number instance
3

%-------------------------------------------------------------------------
# Indexing and Slicing: __getitem__ and __setitem__
>>> class Indexer:
        def __getitem__(self, index):
            return index ** 2
>>> X = Indexer()
>>> X[2] # X[i] calls X.__getitem__(i)
4
>>> for i in range(5):
        print(X[i], end=' ') # Runs __getitem__(X, i) each time
0 1 4 9 16
'''__getitem__ method work for slicing(X[2:4]) and indexing (X[2])'''
>>> class Indexer:
        data = [5, 6, 7, 8, 9]
        def __getitem__(self, index): # Called for index or slice
            print('getitem:', index)
            return self.data[index] # Perform index or slice
>>> X = Indexer()
>>> X[0] # Indexing sends __getitem__ an integer
getitem: 0
5
>>> X[1]
getitem: 1
6
>>> X[-1]
getitem: −1
9
>>> X[2:4] # Slicing sends __getitem__ a slice object
getitem: slice(2, 4, None)
[7, 8]
>>> X[1:]
getitem: slice(1, None, None)
[6, 7, 8, 9]
>>> X[:-1]
getitem: slice(None, −1, None)
[5, 6, 7, 8]
>>> X[::2]
getitem: slice(None, None, 2)
[5, 7, 9]

#Change the value of an index: __setitem__
class IndexSetter:
    def __setitem__(self, index, value): # Intercept index or slice assignment
    ...
        self.data[index] = value # Assign index or slice

# the operator __index__ is not for indexing; it returns an integer value
>>> class C:
        def __index__(self):
            return 255
>>> X = C()
>>> hex(X) # Integer value
'0xff'
>>> bin(X)
'0b11111111'
>>> oct(X)
'0o377'
>>> ('C' * 256)[255]
'C'
>>> ('C' * 256)[X] # As index (not X[i]) --> remember that X is a C() object that returns a 255 value
'C'
>>> ('C' * 256)[X:] # As index (not X[i:])
'C'

# __getitem__ also works as a itrator (index irteration):

>>> class StepperIndex:
        def __getitem__(self, i):
            return self.data[i]
>>> X = StepperIndex() # X is a StepperIndex object
>>> X.data = "Spam"
>>> X[1] # Indexing calls __getitem__
'p'
>>> for item in X: # for loops call __getitem__
        print(item, end=' ') # for indexes items 0..N
S p a m

''' Any type of iterator (membership, list comprehension, for ,map, list and tuple, etc) call __getitem__ automatically:'''

>>> 'p' in X # All call __getitem__ too
True
>>> [c for c in X] # List comprehension
['S', 'p', 'a', 'm']
>>> list(map(str.upper, X)) # map calls (use list() in 3.X)
['S', 'P', 'A', 'M']
>>> (a, b, c, d) = X # Sequence assignments
>>> a, c, d
('S', 'a', 'm')
>>> list(X), tuple(X), ''.join(X) # And so on...
(['S', 'p', 'a', 'm'], ('S', 'p', 'a', 'm'), 'Spam')
>>> X
<__main__.StepperIndex object at 0x000000000297B630>

'''In practice, this technique can be used to create objects that provide a sequence interface
and to add logic to built-in sequence type operations'''
%----------------------------------------------------------------------------------


In [ ]:
# ireatable objects: __iter__ and __next__
'''In practice the operator __iter__ is preferred to __getitem__ : iter support a more general iteration context
   By setting __iter__ python repeatedly calls the iterators object __next__ to produce items until StopIteration protocol.'''
# File squares.py
class Squares:
    def __init__(self, start, stop): # Save state when created
        self.value = start - 1
        self.stop = stop
    def __iter__(self): # Get iterator object on iter
        return self
    def __next__(self): # Return a square on each iteration
        if self.value == self.stop: # Also called by next built-in
            raise StopIteration
        self.value += 1
        return self.value ** 2

# when imported:
% python
>>> from squares import Squares
>>> for i in Squares(1, 5): # for calls iter, which calls __iter__
        print(i, end=' ') # Each iteration calls __next__
1 4 9 16 25

# Manual iteration:
>>> X = Squares(1, 5) # Iterate manually: what loops do
>>> I = iter(X) # iter calls __iter__
>>> next(I) # next calls __next__ (in 3.X)
1
>>> next(I)
4
...more omitted...
>>> next(I)
25
>>> next(I) # Can catch this in try statement
StopIteration

# __iter__ doesnt work for indexing; Only if is coneverted into qa list or any sequence:
>>> X = Squares(1, 5)
>>> X[1]
TypeError: 'Squares' object does not support indexing
>>> list(X)[1] # a list (object) transformation of the object was needed for indexing -> __getitem__ is within list object!
4

'''remember that  __iter__ protocol is exhaustive: designed for a single traversal'''
>>> X = Squares(1, 5) # Make an iterable with state
>>> [n for n in X] # Exhausts items: __iter__ returns self
[1, 4, 9, 16, 25]
>>> [n for n in X] # Now it's empty: __iter__ returns same self
[]

>>> X = Squares(1, 5)
>>> tuple(X), tuple(X) # Iterator exhausted in second tuple()
((1, 4, 9, 16, 25), ())

# you need to make a new instance for each iteration to avopid this problem:
>>> [n for n in Squares(1, 5)] # Make a new iterable object
[1, 4, 9, 16, 25]
>>> list(Squares(1, 3)) # A new object for each new __iter__ call
[1, 4, 9]


'''IMPORTANT: By contrast, other objects suport multiple scans as range, lists , strings and other built-ins'''


In [ ]:
'''Classes vs Generators: In simplier tasks generators (even loops and list comprehension) perform better. Classes do beter incomplex iterations'''

>>> def gsquares(start, stop): # same iterator using generator funct
        for i in range(start, stop + 1):
            yield i ** 2
>>> for i in gsquares(1, 5): # same iteration using for
        print(i, end=' ')
1 4 9 16 25
>>> for i in (x ** 2 for x in range(1, 6)): # same iteration using comprehension + for
        print(i, end=' ')
1 4 9 16 25
>>> [x ** 2 for x in range(1, 6)] # Using list comprehension
[1, 4, 9, 16, 25]




In [ ]:
# __iter__ + yield : you can skip __iter__ buy using yield: yield calls __iter__ automatically and __next__

'''Review on generators funct(yield): '''
>>> def gen(x):
        for i in range(x): yield i ** 2
>>> G = gen(5) # Create a generator with __iter__ and __next__
>>> G.__iter__() == G # Both methods exist on the same object
True
>>> I = iter(G) # Runs __iter__: generator returns itself
>>> next(I), next(I) # Runs __next__ (next in 2.X)
(0, 1)
>>> list(gen(5)) # Iteration contexts automatically run iter and next
[0, 1, 4, 9, 16]
% ------------------------------------------------------------------------------
'''A iterator using __iter__ operator and skipping __next__ operator:'''
# File squares_yield.py
class Squares: # __iter__ + yield generator
    def __init__(self, start, stop): 
        self.start = start
        self.stop = stop
    def __iter__(self):
        for value in range(self.start, self.stop + 1):
            yield value ** 2 # __next__ is automatic/implied

>>> from squares_yield import Squares
>>> for i in Squares(1, 5): print(i, end=' ') # iterator objecyt __iter__ is invoked as usual and __next__ come autom. thx to yield
1 4 9 16 25

# We can call the iter() object without setting __next__ thx to yield:
>>> S = Squares(1, 5) # Runs __init__: class saves instance state
>>> S
<squares_yield.Squares object at 0x000000000294B630>
>>> I = iter(S) # Runs __iter__: returns a generator
>>> I
<generator object __iter__ at 0x00000000029A8CF0>
>>> next(I)
1
>>> next(I) # Runs generator's __next__ --> without creting the attribute __next__ !!
4
...etc...
>>> next(I) # Generator has both instance and local scope state
StopIteration
%------------------------------------------------------------------------
# Also we can skip __iter__ and __next__ by usigng yield within a generator attribute attribute X.gen():
class Squares: # Non __iter__ equivalent (squares_manual.py)
    def __init__(...):
        ...
    def gen(self):
        for value in range(self.start, self.stop + 1):
            yield value ** 2 # yield calls automatically __iter__ and __next__
% python
>>> from squares_manual import Squares
>>> for i in Squares(1, 5).gen(): print(i, end=' ')
...same results...
>>> S = Squares(1, 5)
>>> I = iter(S.gen()) # Call generator manually for iterable/iterator
>>> next(I)
...same results...

%-------------------------------------------------------------------------------------------

# __iter__ + yield supports multiple iterators:
% python
>>> from squares_yield import Squares # Using the __iter__/yield Squares
>>> S = Squares(1, 5)
>>> I = iter(S) # First iteratior I
>>> next(I); next(I)
1
4
>>> J = iter(S) # With yield, multiple iterators automatic --> Second iterator J
>>> next(J)
1
>>> next(I) # I is independent of J: own local state
9

# Doing it without yield: just __iter__
# File squares_nonyield.py
class Squares:
    def __init__(self, start, stop): # Non-yield generator
        self.start = start # Multiscans: extra object
        self.stop = stop
    def __iter__(self):
        return SquaresIter(self.start, self.stop)
class SquaresIter:
    def __init__(self, start, stop):
        self.value = start - 1
        self.stop = stop
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
            self.value += 1
        return self.value ** 2

>>> from squares_nonyield import Squares
>>> for i in Squares(1, 5): print(i, end=' ')
1 4 9 16 25
>>>
>>> S = Squares(1, 5)
>>> I = iter(S)
>>> next(I); next(I)
1
4
>>> J = iter(S) # Multiple iterators without yield
>>> next(J)
1
>>> next(I)
9

__Membership: contains, getitem, and iter__

In [ ]:
'''__contains__ operator provide the class with the in membership operation --> preferred over __iter__ and __getitem__
  under this context'''

# Iters is a classs with all 3 operators:
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i): # Fallback for iteration
        print('get[%s]:' % i, end='') # Also for index, slice
        return self.data[i]
    def __iter__(self): # Preferred for iteration
        print('iter=> ', end='') # Allows only one active iterator
        self.ix = 0
        return self
    def __next__(self):
        print('next:', end='')
        if self.ix == len(self.data): raise StopIteration
            item = self.data[self.ix]
            self.ix += 1
        return item
    def __contains__(self, x): # Preferred for 'in'
        print('contains: ', end='')
        return x in self.data
    next = __next__ # 2.X/3.X compatibility
if __name__ == '__main__':
    X = Iters([1, 2, 3, 4, 5]) # Make instance
    print(3 in X) # Membership
    for i in X: # for loops
        print(i, end=' | ')
    print()
    print([i ** 2 for i in X]) # Other iteration contexts
    print( list(map(bin, X)) )
    I = iter(X) # Manual iteration (what other contexts do)
    while True:
        try:
            print(next(I), end=' @ ')
        except StopIteration:
            break
'''The las code doesnt support multiple scanning. Nevertheless, we can use the yield integration to skip next
   and set multiple iteration'''
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i): # Fallback for iteration
        print('get[%s]:' % i, end='') # Also for index, slice
        return self.data[i]
    def __iter__(self): # Preferred for iteration
        print('iter=> next:', end='') # Allows multiple active iterators
        for x in self.data: # no __next__ to alias to next
            yield x
            print('next:', end='')
    def __contains__(self, x): # Preferred for 'in'
        print('contains: ', end='')
            return x in self.data

'''its output is as follows —the specific __contains__ intercepts membership, the general __iter__ catches other
iteration contexts such that __next__ (whether explicitly coded or implied by yield) is called repeatedly, 
and __getitem__ is never called:'''

contains: True # Doing the in test, python prefers to call _-contain__ to call __iter__
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next: # But iterations keep using __iter__ operand
iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1 @ next:2 @ next:3 @ next:4 @ next:5 @ next:

'''The same output but now commenting the __contain__ method:'''
iter=> next:next:next:True # Now python uses the __iter__ method to test the inoperator
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next: #iterations are done by using __iter__ instead og __getitem__
iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1 @ next:2 @ next:3 @ next:4 @ next:5 @ next:

'''Finally this is what happens using __getitem: commenting on both __contains__ and __iter__'''
get[0]:get[1]:get[2]:True # Now, __getitem__ is used to evaluate in operator
get[0]:1 | get[1]:2 | get[2]:3 | get[3]:4 | get[4]:5 | get[5]: # And also, to carry out iterations
get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:[1, 4, 9, 16, 25]
get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:['0b1', '0b10', '0b11', '0b100','0b101']
get[0]:1 @ get[1]:2 @ get[2]:3 @ get[3]:4 @ get[4]:5 @ get[5]:
# but _-getitem allows our object to be sliced and indexed:
>>> from contains import Iters
>>> X = Iters('spam') # Indexing
>>> X[0] # __getitem__(0)
get[0]:'s'
>>> 'spam'[1:] # Slice syntax
'pam'
>>> 'spam'[slice(1, None)] # Slice object
'pam'
>>> X[1:] # __getitem__(slice(..))
get[slice(1, None, None)]:'pam'
>>> X[:-1]
get[slice(None, −1, None)]:'spa'
>>> list(X) # And iteration too!
iter=> next:next:next:next:next:['s', 'p', 'a', 'm']
                    
'''All in all, __contains__ optimizates membership operator in, __iter__ works better for iteration processes, and __getitem__
  is spacial for slicing and indexing'''

__Attribute Access:__ __ getattr __   and   __ setattr __

In [ ]:
'''__getattr__ : intercept attributes references --> dynamic attribute references'''

class Empty:
    def __getattr__(self, attrname): # On self.undefined
        if attrname == 'age':
            return 40
        else:
            raise AttributeError(attrname)
>>> X = Empty()
>>> X.age # as .age is not a defined attr, __getattr__ takes it and looks for it inside its clauses
40   # now age looks like a real attribute by returning a real value (40) --> dynamically computed
>>> X.name
...error text omitted... # The attribute error rises when __getattr__ cannot handle catching the ttribute.
AttributeError: name

In [ ]:
'''__setattr__: intercepts all forms of object.attribute = value, which is, self.__setattr__('attr',value)'''
class Accesscontrol:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value + 10 # Not self.name=val or setattr --> itll cause an infinite loop
        else:                                # self.age will call again __setattr__, then the last will call self.age, and ono and on!
            raise AttributeError(attr + ' not allowed')

>>> X = Accesscontrol()
>>> X.age = 40 # Calls __setattr__
>>> X.age
50
>>> X.name = 'Bob'
...text omitted...
AttributeError: name not allowed

'''__delattr__ catches all attributes deletions: del object.attr'''
# it is the same as __setattr_ : you should avoid infinite recursion by using class through __dict__ or a superclass

__String Representation:__ __repr__ and __str__

In [ ]:
'''These methods deal with display formats: The default printing of objects is neither useful nor pretty 
   '''
>>> class adder:
        def __init__(self, value=0):
            self.data = value # Initialize data
        def __add__(self, other):
            self.data += other # Add other in place (bad form?)

>>> x = adder() # Default displays
>>> print(x)
<__main__.adder object at 0x00000000029736D8>
>>> x
<__main__.adder object at 0x00000000029736D8>

'''But coding or inheriting string representation methods allows us to customize the display— as in the following, 
   which defines a __repr__ method in a subclass that returns a string representation for its instances.'''

>>> class addrepr(adder): # Inherit __init__, __add__
        def __repr__(self): # Add string representation
            return 'addrepr(%s)' % self.data # Convert to as-code string

>>> x = addrepr(2) # Runs __init__
>>> x + 1 # Runs __add__ (x.add() better?)
>>> x # Runs __repr__
addrepr(3) # human friendly display !!!
>>> print(x) # Runs __repr__
addrepr(3)
>>> str(x), repr(x) # Runs __repr__ for both
('addrepr(3)', 'addrepr(3)')

''' you can use both __repr__ adn __str__ to differentiate displays depending on the final client:'''

>>> class addboth(adder):
def __str__(self):
    return '[Value: %s]' % self.data # User-friendly string
def __repr__(self):
    return 'addboth(%s)' % self.data # As-code string
'''Both retur strins!! otherwise, youll have an error !!! --> use a str converter if needed'''
>>> x = addboth(4)
>>> x + 1
>>> x # Runs __repr__
addboth(5)
>>> print(x) # Runs __str__
[Value: 5]
>>> str(x), repr(x)
('[Value: 5]', 'addboth(5)')

'''Nested objects (within other objects) have to use __repr__ for they representation:'''
>>> class Printer:
        def __init__(self, val):
            self.val = val
        def __str__(self): # Used for instance itself
            return str(self.val) # Convert to a string result

>>> objs = [Printer(2), Printer(3)] #two Printer objects nested in a list object
>>> for x in objs: print(x) # __str__ run when instance printed --> for looop is taking the object at the top level of the script
# But not when instance is in a list!
2
3
>>> print(objs) # but __str__ cannot deal with printing nested objects:
[<__main__.Printer object at 0x000000000297AB38>, <__main__.Printer obj...etc...>]
>>> objs
[<__main__.Printer object at 0x000000000297AB38>, <__main__.Printer obj...etc...>]

# Then, by changing __str__ by __repr__, we can achieve the same result as above, and also print ested objects:
>>> class Printer:
        def __init__(self, val):
            self.val = val
        def __repr__(self): # __repr__ used by print if no __str__
            return str(self.val) # __repr__ used if echoed or nested

>>> objs = [Printer(2), Printer(3)]
>>> for x in objs: print(x) # No __str__: runs __repr__
2
3
>>> print(objs) # Runs __repr__, not ___str__
[2, 3]
>>> objs
[2, 3]

'''Behind __init__, __repr__ is the secondly most commonly used  operator overloading in python.
   Probably any custom display in python has to do with it'''

__Right-Side and In-Place Uses:__ __radd__ and __iadd__

In [ ]:
'''__radd__ and __iadd__ extend the functionality of __add__ and __sub__'''

# Right-Side Addition --> __radd__
>>> class Adder:
    def __init__(self, value=0):
        self.data = value
    def __add__(self, other):
        return self.data + other
>>> x = Adder(5)
>>> x + 2
7
>>> 2 + x
TypeError: unsupported operand type(s) for +: 'int' and 'Adder' # is not a general adder --> the side matters
        
'''In order to support commutative-style operators, it is necessary to set the __radd__ oprator'''
class Commuter1:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val

>>> from commuter import Commuter1
>>> x = Commuter1(88)
>>> y = Commuter1(99)
>>> x + 1 # __add__: instance + noninstance
add 88 1
89
>>> 1 + y # __radd__: noninstance + instance
radd 99 1
100
>>> x + y # __add__: instance + instance, triggers __radd__
add 88 <commuter.Commuter1 object at 0x00000000029B39E8>
radd 99 88
187

# Or you can use __radd__ as just a reused of __add__;

class Commuter2:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self.__add__(other) # Call __add__ explicitly
class Commuter3:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self + other # Swap order and re-add
class Commuter4:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    __radd__ = __add__ # Alias: cut out the middleman
    
# In-place addition : __iadd__
'''One can use the operator += using either an __iadd__ or __add__. The former is preferred to the latter'''

>>> class Number:
        def __init__(self, val):
            self.val = val
        def __iadd__(self, other): # __iadd__ explicit: x += y
            self.val += other # Usually returns self
            return self
>>> x = Number(5)
>>> x += 1
>>> x += 1
>>> x.val
7
'''This method works too in mutable objects:'''
>>> y = Number([1]) # In-place change faster than +
>>> y += [2]
>>> y += [3]
>>> y.val
[1, 2, 3]

'''Final comment: These operators are common in objects that require somo type of numerical processes, e.g, vectors.
   By contrast, a class like Employee or Button, probably would not.'''


__Call Expressions:__ __ call __

In [ ]:
'''the __call__ method is called when your instance is called. It means, when you use your instance as a function'''
>>> class Callee:
        def __call__(self, *pargs, **kargs): # Intercept instance calls
            print('Called:', pargs, kargs) # Accept arbitrary arguments

>>> C = Callee()
>>> C(1, 2, 3) # C is a callable object
Called: (1, 2, 3) {}
>>> C(1, 2, 3, x=4, y=5)
Called: (1, 2, 3) {'y': 5, 'x': 4}

# All the argument passing modes are supported by __call__:
class C:
    def __call__(self, a, b, c=5, d=6): ... # Normals and defaults
class C:
    def __call__(self, *pargs, **kargs): ... # Collect arbitrary arguments
class C:
    def __call__(self, *pargs, d=6, **kargs): ... # 3.X keyword-only argument
        
#any of the previous 3 classes will support the next argument-pass modes:
X = C()
X(1, 2) # Omit defaults
X(1, 2, 3, 4) # Positionals
X(a=1, b=2, d=4) # Keywords
X(*[1, 2], **dict(c=3, d=4)) # Unpack arbitrary arguments
X(1, *(2,), c=3, **dict(d=4)) # Mixed modes

'''Also, __call__ operator support state information retention:'''}
>>> class Prod:
        def __init__(self, value): # Accept just one argument
            self.value = value # this one wold be the state-information
        def __call__(self, other): 
            return self.value * other # other change from call to call, but self.value remains

>>> x = Prod(2) # "Remembers" 2 in state
>>> x(3) # 3 (passed) * 2 (state)
6
>>> x(4) # 4 (passed) * 2 (state)
8
'''In the previous example __call__ could have benn replaced by a normatl attribiute def() achieving the same results.
   However, __call___ is more useful when has to do with APIs interfaces. Is the third most common operator overloader in Python'''


__Comparisons:__ __ lt __, __ gt __, and Others

In [ ]:
'''classes can define methods to catch all six comparison operators: <, >, <=, >=, ==, and !='''
# grater than (__gt__) and lower than(__lt__)
class C:
    data = 'spam'
    def __gt__(self, other): # 3.X and 2.X version
        return self.data > other
    def __lt__(self, other):
        return self.data < other
X = C()
print(X > 'ham') # True (runs __gt__)
print(X < 'ham') # False (runs __lt__)

# Boolean tests:  __bool__ and __len__

'''Python first tries __bool__ to obtain a direct Boolean value; if that method is missing, Python tries 
   __len__ to infer a truth value from the object’s length (a length of 0 means that the obj is false).'''
>>> class Truth:
        def __bool__(self): return True
>>> X = Truth()
>>> if X: print('yes!')
yes!
>>> class Truth:
        def __bool__(self): return False
>>> X = Truth()
>>> bool(X)
False

>>> class Truth:
    def __len__(self): return 0
>>> X = Truth()
>>> if not X: print('no!')
no!

# Anyway, Python prefers __bool__ to __len__ because its more specific
>>> class Truth:
        def __bool__(self): return True # 3.X tries __bool__ first
        def __len__(self): return 0 # 2.X tries __len__ first
>>> X = Truth()
>>> if X: print('yes!')
yes!

# if any method is defined (empty for a boolean test), the empty object is considered true
>>> class Truth:
        pass
>>> X = Truth()
>>> bool(X)
True

__Object Destruction:__ __ del __

In [ ]:
'''the destructor method __del__, is run automatically when an instance’s space is being reclaimed
(i.e., at “garbage collection” time):'''
>>> class Life:
    def __init__(self, name='unknown'):
        print('Hello ' + name)
        self.name = name
    def live(self):
        print(self.name)
    def __del__(self):
        print('Goodbye ' + self.name)

>>> brian = Life('Brian')
Hello Brian
>>> brian.live()
Brian
>>> brian = 'loretta' #__del__ is automatically called when there is a garbage collection process--> a variable pointing out a differnt object in memory
Goodbye Brian
'''However, destructors are not commonly used in python:
  * Python automatically reclaims memory space when an instance is reclaimed.
  It is better to code termination activities with called methods like shutdown, try/finally statment, and with statment '''

In [ ]:
'''Later in this book:
• Chapter 34 uses __enter__ and __exit__ in with statement context managers.
• Chapter 38 uses the __get__ and __set__ class descriptor fetch/set methods.
• Chapter 40 uses the __new__ object creation method in the context of metaclasses.'''